In [1]:
!pip list

Package             Version    
------------------- -----------
absl-py             0.5.0      
alembic             1.0.1      
asn1crypto          0.24.0     
astor               0.7.1      
async-generator     1.10       
backcall            0.1.0      
bleach              3.0.0      
certifi             2018.8.24  
cffi                1.11.5     
chardet             3.0.4      
conda               4.5.11     
cryptography        2.3.1      
cycler              0.10.0     
cymem               1.31.2     
cytoolz             0.9.0.1    
decorator           4.3.0      
defusedxml          0.5.0      
dill                0.2.8.2    
entrypoints         0.2.3      
gast                0.2.0      
GPUtil              1.3.0      
grpcio              1.15.0     
h5py                2.8.0      
idna                2.7        
ipykernel           5.0.0      
ipython             6.5.0      
ipython-genutils    0.2.0      
ipywidgets          7.4.2      
jedi                0.13.1     
Jinja2  

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os



In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

import keras
from nltk.tokenize import TweetTokenizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [5]:
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
data = df.values.tolist()
print(data[0])

[0, 1467810672, 'Mon Apr 06 22:19:49 PDT 2009', 'NO_QUERY', 'scotthamilton', "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"]


In [6]:
master_data = []
for l in data:
    master_data.append([l[0], l[5]])

In [7]:
random.Random(73).shuffle(master_data)

In [8]:
smaller_data = master_data[:50000]
smaller_data[0]

[4, 'thought the Summer Ball was awesome! Well done everyone ']

In [9]:
tokenizer = TweetTokenizer()
r = tokenizer.tokenize(smaller_data[0][1])
print(r)
counter = 0
for l in smaller_data:
    r = ' '.join([x  for x in tokenizer.tokenize(l[1]) if x[0] != '@'])
    smaller_data[counter][1] = r
    counter +=1
    


['thought', 'the', 'Summer', 'Ball', 'was', 'awesome', '!', 'Well', 'done', 'everyone']


In [10]:
print(smaller_data[0])

[4, 'thought the Summer Ball was awesome ! Well done everyone']


In [11]:
max = 0
for pair in smaller_data:
    strLen = len(pair[1].split())
    if strLen > max:
        max = strLen
print(max)

252


In [12]:

sequence_length = 252


In [13]:
max_features = 2000 # this is the number of words we care about

text = [x[1] for x in smaller_data]
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(text)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(text)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)

#print(label)
y = []
for label in smaller_data:
    if label[0] == 0:
        y.append([1,0,0,0,0])
    else:
        y.append([0,0,0,0,1])


        
X_train, X_test, Y_train, Y_test = np.array(X[:40000]), np.array(X[40000:50000]), np.array(y[:40000]), np.array(y[40000:50000])

print("test set size " + str(len(X_train)))
print("test set size " + str(len(X_test)))
print("test set size " + str(len(Y_train)))
print("test set size " + str(len(Y_test)))



test set size 40000
test set size 10000
test set size 40000
test set size 10000


In [14]:
embedding_dim = 200 # We uses 300 here
num_filters = 100

inputs = Input(shape=(sequence_length,), dtype='int32')

# use a random embedding for the text
embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=sequence_length)(inputs)

reshape = Reshape((sequence_length, embedding_dim, 1))(embedding_layer)

# Note the relu activation which we specifically mentions
# He also uses an l2 constraint of 3
# Also, note that the convolution window acts on the whole 200 dimensions - that's important
conv_0 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)

# perform max pooling on each of the convoluations
maxpool_0 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2)

# concat and flatten
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)

# do dropout and predict
dropout = Dropout(0.5)(flatten)
output = Dense(5, activation='softmax')(dropout)

In [15]:
model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 252)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 252, 200)     400000      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 252, 200, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 250, 1, 100)  60100       reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [16]:
batch_size = 32 
history = model.fit(X_train, Y_train, epochs=15, batch_size=batch_size, verbose=1, validation_split=0.1, shuffle=True)

Train on 36000 samples, validate on 4000 samples
Epoch 1/15
36000/36000 [==============================] - 21s 593us/step - loss: 0.8184 - acc: 0.4993 - val_loss: 0.7038 - val_acc: 0.5040
Epoch 2/15
36000/36000 [==============================] - 13s 370us/step - loss: 0.7157 - acc: 0.5716 - val_loss: 0.6890 - val_acc: 0.6580
Epoch 3/15
36000/36000 [==============================] - 13s 370us/step - loss: 0.7112 - acc: 0.6984 - val_loss: 0.7382 - val_acc: 0.7065
Epoch 4/15
36000/36000 [==============================] - 13s 364us/step - loss: 0.7064 - acc: 0.7314 - val_loss: 0.6668 - val_acc: 0.7445
Epoch 5/15
36000/36000 [==============================] - 13s 361us/step - loss: 0.7006 - acc: 0.7448 - val_loss: 0.7207 - val_acc: 0.7372
Epoch 6/15
36000/36000 [==============================] - 13s 357us/step - loss: 0.6926 - acc: 0.7539 - val_loss: 0.6998 - val_acc: 0.7542
Epoch 7/15
36000/36000 [==============================] - 13s 360us/step - loss: 0.6950 - acc: 0.7542 - val_loss: 0.6

In [17]:
y_hat = model.predict(X_test)

In [18]:
accuracy_score(list(map(lambda x: np.argmax(x), Y_test)), list(map(lambda x: np.argmax(x), y_hat)))

0.7536

In [19]:
confusion_matrix(list(map(lambda x: np.argmax(x), Y_test)), list(map(lambda x: np.argmax(x), y_hat)))

array([[3318, 1586],
       [ 878, 4218]])

In [20]:
# serialize model to JSON
model_json = model.to_json()
with open("CnnModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("CnnModel.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
result = model.predict(np.array([X_train[0]]))

In [22]:
print(result)

[[1.01171605e-01 1.11224425e-08 1.06154170e-08 9.53598356e-09
  8.98828387e-01]]


In [24]:
print(Y_train[0])

[0 0 0 0 1]
